In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
opentripmap = pd.read_csv("opentripmap.csv")
opentripmap

,id,xid,name,longitude,latitude,distance,rate,osm,kinds,country
0,3818906,N3429300976,Гостиница «Бристоль»,38.934727,47.211983,NaN,7,node/3429300976,"accomodations,other_hotels",Россия
1,3982337,N4549651178,"амакс курорт """"старая русса""""",31.367519,57.985519,NaN,7,node/4549651178,"accomodations,resorts",Россия
2,3994795,N6152315069,порт на невском,30.353550,59.933083,NaN,7,node/6152315069,"accomodations,hostels,guest_houses",Россия
3,9522713,R6938164,Тихий океан,131.879669,43.117100,NaN,7,relation/6938164,"architecture,historic_architecture,accomodatio...",Россия
4,9595921,R10572941,"hotel metropol palace, a luxury collection hotel",20.473774,44.806789,NaN,7,relation/10572941,"accomodations,other_hotels",Unknown
...,...,...,...,...,...,...,...,...,...,...
2217,9580096,W591351418,93hotel.ru,37.250084,44.986610,11430.177488,1,way/591351418,"accomodations,other_hotels",Россия
2218,1770034,N4315875189,Ванатур,37.255215,44.988403,11431.631234,1,node/4315875189,"guest_houses,accomodations",Россия
2219,9580086,W591347445,отель идеал,37.247829,44.985825,11435.714678,2,way/591347445,"accomodations,other_hotels",Россия
2220,9580090,W591347441,Кронос,37.247273,44.985680,11442.261144,1,way/591347441,"accomodations,other_hotels",Россия


Согласно [документации OpenTripMap API](https://dev.opentripmap.org/docs):
- osm - идентификатор OpenStreetMap объекта
- xid - Уникальный идентификатор объекта в системе OpenTripMap

Для нашего проекта они не имеют бизнес-ценности, поэтому спокойно удаляем


In [18]:
opentripmap = opentripmap.drop(columns=['osm'])
opentripmap = opentripmap.drop(columns=['xid'])

Рассмотрим отели и гостиницы, у которых неизвестно страна расположения 

In [19]:
opentripmap['country'] = opentripmap['country'].replace("Unknown", np.nan)
opentripmap

,id,name,longitude,latitude,distance,rate,kinds,country
0,3818906,Гостиница «Бристоль»,38.934727,47.211983,NaN,7,"accomodations,other_hotels",Россия
1,3982337,"амакс курорт """"старая русса""""",31.367519,57.985519,NaN,7,"accomodations,resorts",Россия
2,3994795,порт на невском,30.353550,59.933083,NaN,7,"accomodations,hostels,guest_houses",Россия
3,9522713,Тихий океан,131.879669,43.117100,NaN,7,"architecture,historic_architecture,accomodatio...",Россия
4,9595921,"hotel metropol palace, a luxury collection hotel",20.473774,44.806789,NaN,7,"accomodations,other_hotels",NaN
...,...,...,...,...,...,...,...,...
2217,9580096,93hotel.ru,37.250084,44.986610,11430.177488,1,"accomodations,other_hotels",Россия
2218,1770034,Ванатур,37.255215,44.988403,11431.631234,1,"guest_houses,accomodations",Россия
2219,9580086,отель идеал,37.247829,44.985825,11435.714678,2,"accomodations,other_hotels",Россия
2220,9580090,Кронос,37.247273,44.985680,11442.261144,1,"accomodations,other_hotels",Россия


In [20]:
missed=opentripmap.isnull().sum()
missed

id             0
name         273
longitude      0
latitude       0
distance     218
rate           0
kinds          0
country      765
dtype: int64

Всего было найдено 765 строк, где не было заполнено страны. Выясним, координаты каких отелей находятся в пределах России
Если координата отеля в пределах России, в столбец country запишем Russia, иначе Other   

In [21]:
def is_in_russia(latitude, longitude):
    if 41.0 <= latitude <= 82.0 and 19.0 <= longitude <= 180.0:
        return True
    else:
        return False

mask = opentripmap['country'].isna()

opentripmap.loc[mask, 'country'] = opentripmap[mask].apply(
    lambda row: "Россия" if is_in_russia(row['latitude'], row['longitude']) else "Other",
    axis=1
)

In [22]:
opentripmap['country'] = opentripmap['country'].replace("Other", np.nan)
opentripmap

,id,name,longitude,latitude,distance,rate,kinds,country
0,3818906,Гостиница «Бристоль»,38.934727,47.211983,NaN,7,"accomodations,other_hotels",Россия
1,3982337,"амакс курорт """"старая русса""""",31.367519,57.985519,NaN,7,"accomodations,resorts",Россия
2,3994795,порт на невском,30.353550,59.933083,NaN,7,"accomodations,hostels,guest_houses",Россия
3,9522713,Тихий океан,131.879669,43.117100,NaN,7,"architecture,historic_architecture,accomodatio...",Россия
4,9595921,"hotel metropol palace, a luxury collection hotel",20.473774,44.806789,NaN,7,"accomodations,other_hotels",Россия
...,...,...,...,...,...,...,...,...
2217,9580096,93hotel.ru,37.250084,44.986610,11430.177488,1,"accomodations,other_hotels",Россия
2218,1770034,Ванатур,37.255215,44.988403,11431.631234,1,"guest_houses,accomodations",Россия
2219,9580086,отель идеал,37.247829,44.985825,11435.714678,2,"accomodations,other_hotels",Россия
2220,9580090,Кронос,37.247273,44.985680,11442.261144,1,"accomodations,other_hotels",Россия


In [23]:
opentripmap.isnull().sum()

id             0
name         273
longitude      0
latitude       0
distance     218
rate           0
kinds          0
country        0
dtype: int64

Выяснилось, что все отели со страной Unknown находятся в России. Продолжаем работу 

Удалим все строки, где названия отелей не заполнены

In [24]:
opentripmap.dropna(subset=['name'], inplace=True)
opentripmap

,id,name,longitude,latitude,distance,rate,kinds,country
0,3818906,Гостиница «Бристоль»,38.934727,47.211983,NaN,7,"accomodations,other_hotels",Россия
1,3982337,"амакс курорт """"старая русса""""",31.367519,57.985519,NaN,7,"accomodations,resorts",Россия
2,3994795,порт на невском,30.353550,59.933083,NaN,7,"accomodations,hostels,guest_houses",Россия
3,9522713,Тихий океан,131.879669,43.117100,NaN,7,"architecture,historic_architecture,accomodatio...",Россия
4,9595921,"hotel metropol palace, a luxury collection hotel",20.473774,44.806789,NaN,7,"accomodations,other_hotels",Россия
...,...,...,...,...,...,...,...,...
2217,9580096,93hotel.ru,37.250084,44.986610,11430.177488,1,"accomodations,other_hotels",Россия
2218,1770034,Ванатур,37.255215,44.988403,11431.631234,1,"guest_houses,accomodations",Россия
2219,9580086,отель идеал,37.247829,44.985825,11435.714678,2,"accomodations,other_hotels",Россия
2220,9580090,Кронос,37.247273,44.985680,11442.261144,1,"accomodations,other_hotels",Россия


Рассмотрим столбец distance. Параметр distance указывает расстояние в метрах от выбранной точки во время запроса по радиусу. То есть нам известно расстояние части отелей от центра города. Это полезная характеристика и важный аспект при ценоформировании. Оставляем этот столбец. Однако стоит учитывать, что часть отелей были получены не методом radius, а с помощью bbox, поэтому множество строк оставлены с незаполненный параметром

In [25]:
opentripmap.isnull().sum()

id             0
name           0
longitude      0
latitude       0
distance     218
rate           0
kinds          0
country        0
dtype: int64

Заполняем незаполненные значения отрицательным числом -1

In [26]:
opentripmap['distance'] = opentripmap['distance'].fillna(-1)
opentripmap

,id,name,longitude,latitude,distance,rate,kinds,country
0,3818906,Гостиница «Бристоль»,38.934727,47.211983,-1.000000,7,"accomodations,other_hotels",Россия
1,3982337,"амакс курорт """"старая русса""""",31.367519,57.985519,-1.000000,7,"accomodations,resorts",Россия
2,3994795,порт на невском,30.353550,59.933083,-1.000000,7,"accomodations,hostels,guest_houses",Россия
3,9522713,Тихий океан,131.879669,43.117100,-1.000000,7,"architecture,historic_architecture,accomodatio...",Россия
4,9595921,"hotel metropol palace, a luxury collection hotel",20.473774,44.806789,-1.000000,7,"accomodations,other_hotels",Россия
...,...,...,...,...,...,...,...,...
2217,9580096,93hotel.ru,37.250084,44.986610,11430.177488,1,"accomodations,other_hotels",Россия
2218,1770034,Ванатур,37.255215,44.988403,11431.631234,1,"guest_houses,accomodations",Россия
2219,9580086,отель идеал,37.247829,44.985825,11435.714678,2,"accomodations,other_hotels",Россия
2220,9580090,Кронос,37.247273,44.985680,11442.261144,1,"accomodations,other_hotels",Россия


In [27]:
opentripmap.to_csv('opentripmap_cleaned.csv', index=False)
opentripmap_cleaned = pd.read_csv("opentripmap_cleaned.csv")

,id,name,longitude,latitude,distance,rate,kinds,country
0,3818906,Гостиница «Бристоль»,38.934727,47.211983,-1.000000,7,"accomodations,other_hotels",Россия
1,3982337,"амакс курорт """"старая русса""""",31.367519,57.985519,-1.000000,7,"accomodations,resorts",Россия
2,3994795,порт на невском,30.353550,59.933083,-1.000000,7,"accomodations,hostels,guest_houses",Россия
3,9522713,Тихий океан,131.879669,43.117100,-1.000000,7,"architecture,historic_architecture,accomodatio...",Россия
4,9595921,"hotel metropol palace, a luxury collection hotel",20.473774,44.806789,-1.000000,7,"accomodations,other_hotels",Россия
...,...,...,...,...,...,...,...,...
1944,9580096,93hotel.ru,37.250084,44.986610,11430.177488,1,"accomodations,other_hotels",Россия
1945,1770034,Ванатур,37.255215,44.988403,11431.631234,1,"guest_houses,accomodations",Россия
1946,9580086,отель идеал,37.247829,44.985825,11435.714678,2,"accomodations,other_hotels",Россия
1947,9580090,Кронос,37.247273,44.985680,11442.261144,1,"accomodations,other_hotels",Россия


In [29]:
opentripmap_cleaned

,id,name,longitude,latitude,distance,rate,kinds,country
0,3818906,Гостиница «Бристоль»,38.934727,47.211983,-1.000000,7,"accomodations,other_hotels",Россия
1,3982337,"амакс курорт """"старая русса""""",31.367519,57.985519,-1.000000,7,"accomodations,resorts",Россия
2,3994795,порт на невском,30.353550,59.933083,-1.000000,7,"accomodations,hostels,guest_houses",Россия
3,9522713,Тихий океан,131.879669,43.117100,-1.000000,7,"architecture,historic_architecture,accomodatio...",Россия
4,9595921,"hotel metropol palace, a luxury collection hotel",20.473774,44.806789,-1.000000,7,"accomodations,other_hotels",Россия
...,...,...,...,...,...,...,...,...
1944,9580096,93hotel.ru,37.250084,44.986610,11430.177488,1,"accomodations,other_hotels",Россия
1945,1770034,Ванатур,37.255215,44.988403,11431.631234,1,"guest_houses,accomodations",Россия
1946,9580086,отель идеал,37.247829,44.985825,11435.714678,2,"accomodations,other_hotels",Россия
1947,9580090,Кронос,37.247273,44.985680,11442.261144,1,"accomodations,other_hotels",Россия
